In [1]:
from haystack.document_stores import InMemoryDocumentStore
document_store = InMemoryDocumentStore(use_bm25=True)

In [2]:
from haystack.nodes import TextConverter, PDFToTextConverter, DocxToTextConverter, PreProcessor
doc_dir = r"C:/Users/avkuz/TTRPG Coding/data/rules.txt"
converter = TextConverter(remove_numeric_tables=True, valid_languages=["en"])
doc_txt = converter.convert(file_path=doc_dir, meta=None)[0]


# preprocessing

In [3]:
from haystack.nodes import PreProcessor


# This is a default usage of the PreProcessor.
# Here, it performs cleaning of consecutive whitespaces
# and splits a single large document into smaller documents.
# Each document is up to 1000 words long and document breaks cannot fall in the middle of sentences
# Note how the single document passed into the document gets split into 5 smaller documents

preprocessor = PreProcessor(
    clean_empty_lines=True,
    clean_whitespace=True,
    clean_header_footer=True,
    split_by="word",
    split_length=1000,
    split_respect_sentence_boundary=True,
)
docs_default = preprocessor.process([doc_txt])
print(f"n_docs_input: 1\nn_docs_output: {len(docs_default)}")

Preprocessing:   0%|          | 0/1 [00:00<?, ?docs/s]

n_docs_input: 1
n_docs_output: 36


In [4]:
import os
from haystack.pipelines.standard_pipelines import TextIndexingPipeline

files_to_index = [doc_dir]
indexing_pipeline = TextIndexingPipeline(document_store)
indexing_pipeline.run_batch(file_paths=files_to_index)

Converting files:   0%|          | 0/1 [00:00<?, ?it/s]

Preprocessing:   0%|          | 0/1 [00:00<?, ?docs/s]

We found one or more sentences whose word count is higher than the split length.


Updating BM25 representation...:   0%|          | 0/192 [00:00<?, ? docs/s]

{'documents': [<Document: {'content': "\nAs your character goes on adventures and overcomes challenges, he or she gains experience, represented by experience points. A character who reaches a specified experience point total advances in capability. This advancement is called gaining a level.\n\nWhen your character gains a level, his or her class often grants additional features, as detailed in the class description. Some of these features allow you to increase your ability scores, either increasing two scores by 1 each or increasing one score by 2. You can't increase an ability score above 20. In addition, every character's proficiency bonus increases at certain levels.\n\nEach time you gain a level, you gain 1 additional Hit Die. Roll that Hit Die, add your Constitution modifier to the roll, and add the total (minimum of 1) to your hit point maximum. Alternatively, you can use the fixed value shown in your class entry, which is the average result of the die roll (rounded up).\n\nWhen 

# bert

In [14]:
from haystack.document_stores import InMemoryDocumentStore
document_store = InMemoryDocumentStore(use_bm25=True)

In [15]:
from haystack.nodes import TextConverter, PDFToTextConverter, DocxToTextConverter, PreProcessor
doc_dir = r"C:/Users/avkuz/TTRPG Coding/data/rules.txt"
converter = TextConverter(remove_numeric_tables=True, valid_languages=["en"])
doc_txt = converter.convert(file_path=doc_dir, meta=None)[0]


In [16]:
from haystack.nodes import PreProcessor


# This is a default usage of the PreProcessor.
# Here, it performs cleaning of consecutive whitespaces
# and splits a single large document into smaller documents.
# Each document is up to 1000 words long and document breaks cannot fall in the middle of sentences
# Note how the single document passed into the document gets split into 5 smaller documents

preprocessor = PreProcessor(
    clean_empty_lines=True,
    clean_whitespace=True,
    clean_header_footer=True,
    split_by="word",
    split_length=1000,
    split_respect_sentence_boundary=True,
)
docs_default = preprocessor.process([doc_txt])
print(f"n_docs_input: 1\nn_docs_output: {len(docs_default)}")

Preprocessing:   0%|          | 0/1 [00:00<?, ?docs/s]

n_docs_input: 1
n_docs_output: 36


In [17]:
import os
from haystack.pipelines.standard_pipelines import TextIndexingPipeline

files_to_index = [doc_dir]
indexing_pipeline = TextIndexingPipeline(document_store)
indexing_pipeline.run_batch(file_paths=files_to_index)

Converting files:   0%|          | 0/1 [00:00<?, ?it/s]

Preprocessing:   0%|          | 0/1 [00:00<?, ?docs/s]

We found one or more sentences whose word count is higher than the split length.


Updating BM25 representation...:   0%|          | 0/192 [00:00<?, ? docs/s]

{'documents': [<Document: {'content': "\nAs your character goes on adventures and overcomes challenges, he or she gains experience, represented by experience points. A character who reaches a specified experience point total advances in capability. This advancement is called gaining a level.\n\nWhen your character gains a level, his or her class often grants additional features, as detailed in the class description. Some of these features allow you to increase your ability scores, either increasing two scores by 1 each or increasing one score by 2. You can't increase an ability score above 20. In addition, every character's proficiency bonus increases at certain levels.\n\nEach time you gain a level, you gain 1 additional Hit Die. Roll that Hit Die, add your Constitution modifier to the roll, and add the total (minimum of 1) to your hit point maximum. Alternatively, you can use the fixed value shown in your class entry, which is the average result of the die roll (rounded up).\n\nWhen 

In [18]:
from haystack.nodes import BM25Retriever

retriever = BM25Retriever(document_store=document_store)

In [19]:
from haystack.nodes import FARMReader

reader = FARMReader(model_name_or_path="deepset/bert-large-uncased-whole-word-masking-squad2", use_gpu=True)

Downloading:   0%|          | 0.00/540 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/19.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [20]:
from haystack.pipelines import ExtractiveQAPipeline

pipe = ExtractiveQAPipeline(reader, retriever)

In [21]:
prediction = pipe.run(
    query="How to make an attack roll?",
    params={
        "Retriever": {"top_k": 10},
        "Reader": {"top_k": 5}
    }
)
from haystack.utils import print_answers

print_answers(
    prediction,
    details="minimum" ## Choose from `minimum`, `medium`, and `all`
)

Inferencing Samples:   0%|          | 0/1 [00:00<?, ? Batches/s]

'Query: How to make an attack roll?'
'Answers:'
[   {   'answer': 'roll a d20 and add the appropriate modifiers',
        'context': 'r the attack hits or misses. To make an attack roll, roll '
                   'a d20 and add the appropriate modifiers. If the total of '
                   'the roll plus modifiers equals or e'},
    {   'answer': 'using a weapon with which you lack proficiency',
        'context': 'u make with that weapon. If you make an attack roll using '
                   'a weapon with which you lack proficiency, you do not add '
                   'your proficiency bonus to the attac'},
    {   'answer': 'it uses whatever modifier is provided in its stat block',
        'context': 'When a monster makes an attack roll, it uses whatever '
                   'modifier is provided in its stat block.\n'
                   'Ability Modifier.\n'
                   '\n'
                   'The ability modifier used for a melee'},
    {   'answer': 'you roll a second d20

In [22]:
prediction = pipe.run(
    query="What is the attack modifier for a rapier?",
    params={
        "Retriever": {"top_k": 10},
        "Reader": {"top_k": 5}
    }
)
from haystack.utils import print_answers

print_answers(
    prediction,
    details="minimum" ## Choose from `minimum`, `medium`, and `all`
)

Inferencing Samples:   0%|          | 0/1 [00:00<?, ? Batches/s]

'Query: What is the attack modifier for a rapier?'
'Answers:'
[   {   'answer': 'Dexterity',
        'context': 'A weapon that has the finesse property, such as a rapier, '
                   'can use your Dexterity modifier instead.\n'
                   'For attacks with ranged weapons, use your Dexterity'},
    {   'answer': 'Strength',
        'context': 'lity Modifier.\n'
                   '\n'
                   'The ability modifier used for a melee weapon attack is '
                   'Strength, and the ability modifier used for a ranged '
                   'weapon attack is Dexterity'},
    {   'answer': 'Strength or Dexterity',
        'context': 'ing an attack with a finesse weapon, you use your choice '
                   'of your Strength or Dexterity modifier for the attack and '
                   'damage rolls. You must use the same'},
    {   'answer': 'negative',
        'context': 'ty modifier to the damage of the bonus attack, unless that '
                   'mo

In [23]:
prediction = pipe.run(
    query="How many arrows does the quiver hold?",
    params={
        "Retriever": {"top_k": 10},
        "Reader": {"top_k": 5}
    }
)
from haystack.utils import print_answers

print_answers(
    prediction,
    details="minimum" ## Choose from `minimum`, `medium`, and `all`
)

Inferencing Samples:   0%|          | 0/1 [00:00<?, ? Batches/s]

'Query: How many arrows does the quiver hold?'
'Answers:'
[   {   'answer': '20',
        'context': 'vial)\t25 gp\t1 lb.\n'
                   "Alchemist's fire (flask)\t50 gp\t1 lb.\n"
                   'Ammunition\n'
                   'Arrows (20)\t1 gp\t1 lb.\n'
                   'Blowgun needles (50)\t1 gp\t1 lb.\n'
                   'Crossbow bolts (20)\t1 gp\t1½ l'},
    {   'answer': 'up to 20 arrows',
        'context': 'uch (described earlier in this section).\n'
                   'Quiver.\n'
                   '\n'
                   'A quiver can hold up to 20 arrows.\n'
                   'Ram, Portable.\n'
                   '\n'
                   'You can use a portable ram to break down doors. W'},
    {   'answer': '14',
        'context': 'til it can breathe again.\n'
                   '\n'
                   'For example, a creature with a Constitution of 14 can hold '
                   'its breath for 3 minutes. If it starts suffocating, it has '
           

In [24]:
prediction = pipe.run(
    query="What can I use Charisma for?",
    params={
        "Retriever": {"top_k": 10},
        "Reader": {"top_k": 5}
    }
)
from haystack.utils import print_answers

print_answers(
    prediction,
    details="minimum" ## Choose from `minimum`, `medium`, and `all`
)

Inferencing Samples:   0%|          | 0/1 [00:00<?, ? Batches/s]

'Query: What can I use Charisma for?'
'Answers:'
[   {   'answer': 'to interact effectively with others',
        'context': 'ess.\n'
                   'Charisma PHB p178[–]\n'
                   '\n'
                   'Charisma measures your ability to interact effectively '
                   'with others. It includes such factors as confidence and '
                   'eloquence, a'},
    {   'answer': 'Sorcerer',
        'context': '3 and Charisma 13\n'
                   'Ranger\tDexterity 13 and Wisdom 13\n'
                   'Rogue\tDexterity 13\n'
                   'Sorcerer\tCharisma 13\n'
                   'Warlock\tCharisma 13\n'
                   'Wizard\tIntelligence 13\n'
                   'Experience Poin'},
    {   'answer': 'Persuasion',
        'context': 'cial graces, or good nature, the DM might ask you to make '
                   'a Charisma (Persuasion) check. Typically, you use '
                   'persuasion when acting in good faith, to f'},
    {   'a

# Roberta

In [1]:
from haystack.document_stores import InMemoryDocumentStore
document_store = InMemoryDocumentStore(use_bm25=True)

In [2]:
from haystack.nodes import TextConverter, PDFToTextConverter, DocxToTextConverter, PreProcessor
doc_dir = r"C:/Users/avkuz/TTRPG Coding/data/rules.txt"
converter = TextConverter(remove_numeric_tables=True, valid_languages=["en"])
doc_txt = converter.convert(file_path=doc_dir, meta=None)[0]


In [3]:
from haystack.nodes import PreProcessor


# This is a default usage of the PreProcessor.
# Here, it performs cleaning of consecutive whitespaces
# and splits a single large document into smaller documents.
# Each document is up to 1000 words long and document breaks cannot fall in the middle of sentences
# Note how the single document passed into the document gets split into 5 smaller documents

preprocessor = PreProcessor(
    clean_empty_lines=True,
    clean_whitespace=True,
    clean_header_footer=True,
    split_by="word",
    split_length=1000,
    split_respect_sentence_boundary=True,
)
docs_default = preprocessor.process([doc_txt])
print(f"n_docs_input: 1\nn_docs_output: {len(docs_default)}")

Preprocessing:   0%|          | 0/1 [00:00<?, ?docs/s]

n_docs_input: 1
n_docs_output: 36


In [4]:
import os
from haystack.pipelines.standard_pipelines import TextIndexingPipeline

files_to_index = [doc_dir]
indexing_pipeline = TextIndexingPipeline(document_store)
indexing_pipeline.run_batch(file_paths=files_to_index)

Converting files:   0%|          | 0/1 [00:00<?, ?it/s]

Preprocessing:   0%|          | 0/1 [00:00<?, ?docs/s]

We found one or more sentences whose word count is higher than the split length.


Updating BM25 representation...:   0%|          | 0/192 [00:00<?, ? docs/s]

{'documents': [<Document: {'content': "\nAs your character goes on adventures and overcomes challenges, he or she gains experience, represented by experience points. A character who reaches a specified experience point total advances in capability. This advancement is called gaining a level.\n\nWhen your character gains a level, his or her class often grants additional features, as detailed in the class description. Some of these features allow you to increase your ability scores, either increasing two scores by 1 each or increasing one score by 2. You can't increase an ability score above 20. In addition, every character's proficiency bonus increases at certain levels.\n\nEach time you gain a level, you gain 1 additional Hit Die. Roll that Hit Die, add your Constitution modifier to the roll, and add the total (minimum of 1) to your hit point maximum. Alternatively, you can use the fixed value shown in your class entry, which is the average result of the die roll (rounded up).\n\nWhen 

In [5]:
from haystack.nodes import BM25Retriever

retriever = BM25Retriever(document_store=document_store)

In [7]:
from haystack.nodes import FARMReader

reader = FARMReader(model_name_or_path="deepset/roberta-base-squad2", use_gpu=True)

In [8]:
from haystack.pipelines import ExtractiveQAPipeline

pipe = ExtractiveQAPipeline(reader, retriever)

In [9]:
prediction = pipe.run(
    query="How to make an attack roll?",
    params={
        "Retriever": {"top_k": 10},
        "Reader": {"top_k": 5}
    }
)
from haystack.utils import print_answers

print_answers(
    prediction,
    details="minimum" ## Choose from `minimum`, `medium`, and `all`
)

Inferencing Samples:   0%|          | 0/1 [00:00<?, ? Batches/s]

'Query: How to make an attack roll?'
'Answers:'
[   {   'answer': 'roll a d20 and add the appropriate modifiers',
        'context': 'r the attack hits or misses. To make an attack roll, roll '
                   'a d20 and add the appropriate modifiers. If the total of '
                   'the roll plus modifiers equals or e'},
    {   'answer': 'you roll a second d20',
        'context': 'ity check, a saving throw, or an attack roll. When that '
                   'happens, you roll a second d20 when you make the roll. Use '
                   'the higher of the two rolls if you '},
    {   'answer': 'second d20',
        'context': 'eck, a saving throw, or an attack roll. When that happens, '
                   'you roll a second d20 when you make the roll. Use the '
                   'higher of the two rolls if you have a'},
    {   'answer': 'whatever modifier is provided in its stat block',
        'context': 'When a monster makes an attack roll, it uses whatever '
            

In [10]:
prediction = pipe.run(
    query="What is the attack modifier for a rapier?",
    params={
        "Retriever": {"top_k": 10},
        "Reader": {"top_k": 5}
    }
)
from haystack.utils import print_answers

print_answers(
    prediction,
    details="minimum" ## Choose from `minimum`, `medium`, and `all`
)

Inferencing Samples:   0%|          | 0/1 [00:00<?, ? Batches/s]

'Query: What is the attack modifier for a rapier?'
'Answers:'
[   {   'answer': 'Dexterity',
        'context': 'A weapon that has the finesse property, such as a rapier, '
                   'can use your Dexterity modifier instead.\n'
                   'For attacks with ranged weapons, use your Dexterity'},
    {   'answer': 'negative',
        'context': 'ty modifier to the damage of the bonus attack, unless that '
                   'modifier is negative.\n'
                   '\n'
                   'If either weapon has the thrown property, you can throw '
                   'the weapon, '},
    {   'answer': 'Strength',
        'context': 'lity Modifier.\n'
                   '\n'
                   'The ability modifier used for a melee weapon attack is '
                   'Strength, and the ability modifier used for a ranged '
                   'weapon attack is Dexterity'},
    {   'answer': 'Strength',
        'context': 'and two handaxes. His battleaxe is a melee weapon,

In [11]:
prediction = pipe.run(
    query="How many arrows does the quiver hold?",
    params={
        "Retriever": {"top_k": 10},
        "Reader": {"top_k": 5}
    }
)
from haystack.utils import print_answers

print_answers(
    prediction,
    details="minimum" ## Choose from `minimum`, `medium`, and `all`
)

Inferencing Samples:   0%|          | 0/1 [00:00<?, ? Batches/s]

'Query: How many arrows does the quiver hold?'
'Answers:'
[   {   'answer': '20',
        'context': 'uch (described earlier in this section).\n'
                   'Quiver.\n'
                   '\n'
                   'A quiver can hold up to 20 arrows.\n'
                   'Ram, Portable.\n'
                   '\n'
                   'You can use a portable ram to break down doors. W'},
    {   'answer': '1 gp',
        'context': '\t25 gp\t1 lb.\n'
                   "Alchemist's fire (flask)\t50 gp\t1 lb.\n"
                   'Ammunition\n'
                   'Arrows (20)\t1 gp\t1 lb.\n'
                   'Blowgun needles (50)\t1 gp\t1 lb.\n'
                   'Crossbow bolts (20)\t1 gp\t1½ lb.\n'
                   'Sl'},
    {   'answer': 'twenty',
        'context': 'This wooden case can hold up to twenty crossbow bolts\n'
                   'Case, Map or Scroll.\n'
                   '\n'
                   'This cylindrical leather case can hold up to ten rolled-up '
    

In [13]:
prediction = pipe.run(
    query="What is Charisma used  for?",
    params={
        "Retriever": {"top_k": 10},
        "Reader": {"top_k": 5}
    }
)
from haystack.utils import print_answers

print_answers(
    prediction,
    details="minimum" ## Choose from `minimum`, `medium`, and `all`
)

Inferencing Samples:   0%|          | 0/1 [00:00<?, ? Batches/s]

'Query: What is Charisma used  for?'
'Answers:'
[   {   'answer': 'measures your ability to interact effectively with others',
        'context': 'nce of success.\n'
                   'Charisma PHB p178[–]\n'
                   '\n'
                   'Charisma measures your ability to interact effectively '
                   'with others. It includes such factors as confidence and e'},
    {   'answer': 'to interact effectively with others',
        'context': 'ess.\n'
                   'Charisma PHB p178[–]\n'
                   '\n'
                   'Charisma measures your ability to interact effectively '
                   'with others. It includes such factors as confidence and '
                   'eloquence, a'},
    {   'answer': 'Paladin',
        'context': ' 13\n'
                   'Fighter\tStrength 13 or Dexterity 13\n'
                   'Monk\tDexterity 13 and Wisdom 13\n'
                   'Paladin\tStrength 13 and Charisma 13\n'
                   'Ranger\tDext

# xlm-roberta

In [ ]:
from haystack.document_stores import InMemoryDocumentStore
document_store = InMemoryDocumentStore(use_bm25=True)

In [ ]:
from haystack.nodes import TextConverter, PDFToTextConverter, DocxToTextConverter, PreProcessor
doc_dir = r"C:/Users/avkuz/TTRPG Coding/data/rules.txt"
converter = TextConverter(remove_numeric_tables=True, valid_languages=["en"])
doc_txt = converter.convert(file_path=doc_dir, meta=None)[0]


In [ ]:
from haystack.nodes import PreProcessor


# This is a default usage of the PreProcessor.
# Here, it performs cleaning of consecutive whitespaces
# and splits a single large document into smaller documents.
# Each document is up to 1000 words long and document breaks cannot fall in the middle of sentences
# Note how the single document passed into the document gets split into 5 smaller documents

preprocessor = PreProcessor(
    clean_empty_lines=True,
    clean_whitespace=True,
    clean_header_footer=True,
    split_by="word",
    split_length=1000,
    split_respect_sentence_boundary=True,
)
docs_default = preprocessor.process([doc_txt])
print(f"n_docs_input: 1\nn_docs_output: {len(docs_default)}")

In [ ]:
import os
from haystack.pipelines.standard_pipelines import TextIndexingPipeline

files_to_index = [doc_dir]
indexing_pipeline = TextIndexingPipeline(document_store)
indexing_pipeline.run_batch(file_paths=files_to_index)

In [12]:
from haystack.nodes import BM25Retriever

retriever = BM25Retriever(document_store=document_store)

In [13]:
from haystack.nodes import FARMReader

reader = FARMReader(model_name_or_path="deepset/xlm-roberta-large-squad2", use_gpu=True)

In [14]:
from haystack.pipelines import ExtractiveQAPipeline

pipe = ExtractiveQAPipeline(reader, retriever)

In [15]:
prediction = pipe.run(
    query="How to make an attack roll?",
    params={
        "Retriever": {"top_k": 10},
        "Reader": {"top_k": 5}
    }
)
from haystack.utils import print_answers

print_answers(
    prediction,
    details="minimum" ## Choose from `minimum`, `medium`, and `all`
)

Inferencing Samples:   0%|          | 0/1 [00:00<?, ? Batches/s]

'Query: How to make an attack roll?'
'Answers:'
[   {   'answer': 'roll a d20 and add the appropriate modifiers',
        'context': 'r the attack hits or misses. To make an attack roll, roll '
                   'a d20 and add the appropriate modifiers. If the total of '
                   'the roll plus modifiers equals or e'},
    {   'answer': 'to determine whether the spell effect hits the intended '
                  'target',
        'context': 's require the caster to make an attack roll to determine '
                   'whether the spell effect hits the intended target. Your '
                   'attack bonus with a spell attack equa'},
    {   'answer': 'If you make an attack roll using a weapon with which you '
                  'lack proficiency',
        'context': ' any attack you make with that weapon. If you make an '
                   'attack roll using a weapon with which you lack '
                   'proficiency, you do not add your proficiency bonu'},
    {   'an

In [16]:
prediction = pipe.run(
    query="What is the attack modifier for a rapier?",
    params={
        "Retriever": {"top_k": 10},
        "Reader": {"top_k": 5}
    }
)
from haystack.utils import print_answers

print_answers(
    prediction,
    details="minimum" ## Choose from `minimum`, `medium`, and `all`
)

Inferencing Samples:   0%|          | 0/1 [00:00<?, ? Batches/s]

'Query: What is the attack modifier for a rapier?'
'Answers:'
[   {   'answer': 'Dexterity',
        'context': 'A weapon that has the finesse property, such as a rapier, '
                   'can use your Dexterity modifier instead.\n'
                   'For attacks with ranged weapons, use your Dexterity'},
    {   'answer': 'Strength or Dexterity',
        'context': 'ing an attack with a finesse weapon, you use your choice '
                   'of your Strength or Dexterity modifier for the attack and '
                   'damage rolls. You must use the same'},
    {   'answer': 'Wisdom',
        'context': "e's passive Wisdom (Perception) score, which equals 10 + "
                   "the creature's Wisdom modifier, as well as any other "
                   'bonuses or penalties. If the creature ha'},
    {   'answer': 'Wisdom',
        'context': "e's passive Wisdom (Perception) score, which equals 10 + "
                   "the creature's Wisdom modifier, as well as any other "

In [17]:
prediction = pipe.run(
    query="How many arrows does the quiver hold?",
    params={
        "Retriever": {"top_k": 10},
        "Reader": {"top_k": 5}
    }
)
from haystack.utils import print_answers

print_answers(
    prediction,
    details="minimum" ## Choose from `minimum`, `medium`, and `all`
)

Inferencing Samples:   0%|          | 0/1 [00:00<?, ? Batches/s]

'Query: How many arrows does the quiver hold?'
'Answers:'
[   {   'answer': 'up to 20 arrows',
        'context': 'uch (described earlier in this section).\n'
                   'Quiver.\n'
                   '\n'
                   'A quiver can hold up to 20 arrows.\n'
                   'Ram, Portable.\n'
                   '\n'
                   'You can use a portable ram to break down doors. W'},
    {   'answer': 'Ammunition\nArrows (20)',
        'context': 'b.\n'
                   'Acid (vial)\t25 gp\t1 lb.\n'
                   "Alchemist's fire (flask)\t50 gp\t1 lb.\n"
                   'Ammunition\n'
                   'Arrows (20)\t1 gp\t1 lb.\n'
                   'Blowgun needles (50)\t1 gp\t1 lb.\n'
                   'Crossbow bolts (20)\t'},
    {   'answer': 'twenty',
        'context': 'This wooden case can hold up to twenty crossbow bolts\n'
                   'Case, Map or Scroll.\n'
                   '\n'
                   'This cylindrical leather case can 

In [18]:
prediction = pipe.run(
    query="What can I use Charisma for?",
    params={
        "Retriever": {"top_k": 10},
        "Reader": {"top_k": 5}
    }
)
from haystack.utils import print_answers

print_answers(
    prediction,
    details="minimum" ## Choose from `minimum`, `medium`, and `all`
)

Inferencing Samples:   0%|          | 0/1 [00:00<?, ? Batches/s]

'Query: What can I use Charisma for?'
'Answers:'
[   {   'answer': 'spellcasting ability',
        'context': '\n'
                   '\n'
                   'Bards, paladins, sorcerers, and warlocks use Charisma as '
                   'their spellcasting ability, which helps determine the '
                   'saving throw DCs of spells they cast.'},
    {   'answer': 'measures your ability to interact effectively with others',
        'context': 'nce of success.\n'
                   'Charisma PHB p178[–]\n'
                   '\n'
                   'Charisma measures your ability to interact effectively '
                   'with others. It includes such factors as confidence and e'},
    {   'answer': 'Deception, Intimidation, Performance, Persuasion',
        'context': ', Insight, Medicine, Perception, Survival\n'
                   'Charisma\tDeception, Intimidation, Performance, '
                   'Persuasion\n'
                   '\n'
                   'Sometimes, the DM 

# deberta

In [1]:
from haystack.document_stores import InMemoryDocumentStore
document_store = InMemoryDocumentStore(use_bm25=True)

In [2]:
from haystack.nodes import TextConverter, PDFToTextConverter, DocxToTextConverter, PreProcessor
doc_dir = r"C:/Users/avkuz/TTRPG Coding/data/rules.txt"
converter = TextConverter(remove_numeric_tables=True, valid_languages=["en"])
doc_txt = converter.convert(file_path=doc_dir, meta=None)[0]


In [3]:
from haystack.nodes import PreProcessor


# This is a default usage of the PreProcessor.
# Here, it performs cleaning of consecutive whitespaces
# and splits a single large document into smaller documents.
# Each document is up to 1000 words long and document breaks cannot fall in the middle of sentences
# Note how the single document passed into the document gets split into 5 smaller documents

preprocessor = PreProcessor(
    clean_empty_lines=True,
    clean_whitespace=True,
    clean_header_footer=True,
    split_by="word",
    split_length=1000,
    split_respect_sentence_boundary=True,
)
docs_default = preprocessor.process([doc_txt])
print(f"n_docs_input: 1\nn_docs_output: {len(docs_default)}")

Preprocessing:   0%|          | 0/1 [00:00<?, ?docs/s]

n_docs_input: 1
n_docs_output: 36


In [7]:
import os
from haystack.pipelines.standard_pipelines import TextIndexingPipeline

files_to_index = [doc_dir]
indexing_pipeline = TextIndexingPipeline(document_store)
indexing_pipeline.run_batch(file_paths=files_to_index)

Converting files:   0%|          | 0/1 [00:00<?, ?it/s]

Preprocessing:   0%|          | 0/1 [00:00<?, ?docs/s]

We found one or more sentences whose word count is higher than the split length.


Updating BM25 representation...:   0%|          | 0/192 [00:00<?, ? docs/s]

{'documents': [<Document: {'content': "\nAs your character goes on adventures and overcomes challenges, he or she gains experience, represented by experience points. A character who reaches a specified experience point total advances in capability. This advancement is called gaining a level.\n\nWhen your character gains a level, his or her class often grants additional features, as detailed in the class description. Some of these features allow you to increase your ability scores, either increasing two scores by 1 each or increasing one score by 2. You can't increase an ability score above 20. In addition, every character's proficiency bonus increases at certain levels.\n\nEach time you gain a level, you gain 1 additional Hit Die. Roll that Hit Die, add your Constitution modifier to the roll, and add the total (minimum of 1) to your hit point maximum. Alternatively, you can use the fixed value shown in your class entry, which is the average result of the die roll (rounded up).\n\nWhen 

In [8]:
from haystack.nodes import BM25Retriever

retriever = BM25Retriever(document_store=document_store)

In [9]:
from haystack.nodes import FARMReader

reader = FARMReader(model_name_or_path="deepset/deberta-v3-large-squad2", use_gpu=True)

In [10]:
from haystack.pipelines import ExtractiveQAPipeline

pipe = ExtractiveQAPipeline(reader, retriever)

In [11]:
prediction = pipe.run(
    query="How to make an attack?",
    params={
        "Retriever": {"top_k": 10},
        "Reader": {"top_k": 5}
    }
)
from haystack.utils import print_answers

print_answers(
    prediction,
    details="minimum" ## Choose from `minimum`, `medium`, and `all`
)

Inferencing Samples:   0%|          | 0/1 [00:00<?, ? Batches/s]

'Query: How to make an attack?'
'Answers:'
[   {   'answer': 'roll a d20 and add the appropriate modifiers',
        'context': 'r the attack hits or misses. To make an attack roll, roll '
                   'a d20 and add the appropriate modifiers. If the total of '
                   'the roll plus modifiers equals or e'},
    {   'answer': 'You make the attack roll',
        'context': 'alties or bonuses to your attack roll.\n'
                   '\n'
                   '3. Resolve the attack. You make the attack roll. On a hit, '
                   'you roll damage, unless the particular attack has r'},
    {   'answer': 'weapon is used with two hands',
        'context': 'in parentheses appears with the property-the damage when '
                   'the weapon is used with two hands to make a melee attack.\n'
                   'Improvised Weapons PHB p147[–]\n'
                   '\n'
                   'Som'},
    {   'answer': 'Proficiency with a weapon allows you to add your '

In [12]:
prediction = pipe.run(
    query="What is the attack modifier for the rapier?",
    params={
        "Retriever": {"top_k": 10},
        "Reader": {"top_k": 5}
    }
)
from haystack.utils import print_answers

print_answers(
    prediction,
    details="minimum" ## Choose from `minimum`, `medium`, and `all`
)

Inferencing Samples:   0%|          | 0/1 [00:00<?, ? Batches/s]

'Query: What is the attack modifier for the rapier?'
'Answers:'
[   {   'answer': 'Dexterity',
        'context': 'A weapon that has the finesse property, such as a rapier, '
                   'can use your Dexterity modifier instead.\n'
                   'For attacks with ranged weapons, use your Dexterity'},
    {   'answer': 'finesse',
        'context': 'll and your damage roll when attacking with a melee weapon '
                   'that has the finesse property, such as a dagger or a '
                   'rapier.\n'
                   'Armor Class PHB p177[–]\n'
                   '\n'
                   'Depen'},
    {   'answer': 'Strength or Dexterity',
        'context': 'ing an attack with a finesse weapon, you use your choice '
                   'of your Strength or Dexterity modifier for the attack and '
                   'damage rolls. You must use the same'},
    {   'answer': '+2',
        'context': ' Bob also notes the proficiency bonus for a 1st-level '
         

In [13]:
prediction = pipe.run(
    query="How many arrows does the quiver hold?",
    params={
        "Retriever": {"top_k": 10},
        "Reader": {"top_k": 5}
    }
)
from haystack.utils import print_answers

print_answers(
    prediction,
    details="minimum" ## Choose from `minimum`, `medium`, and `all`
)

Inferencing Samples:   0%|          | 0/1 [00:00<?, ? Batches/s]

'Query: How many arrows does the quiver hold?'
'Answers:'
[   {   'answer': '20',
        'context': 'vial)\t25 gp\t1 lb.\n'
                   "Alchemist's fire (flask)\t50 gp\t1 lb.\n"
                   'Ammunition\n'
                   'Arrows (20)\t1 gp\t1 lb.\n'
                   'Blowgun needles (50)\t1 gp\t1 lb.\n'
                   'Crossbow bolts (20)\t1 gp\t1½ l'},
    {   'answer': '20',
        'context': 'uch (described earlier in this section).\n'
                   'Quiver.\n'
                   '\n'
                   'A quiver can hold up to 20 arrows.\n'
                   'Ram, Portable.\n'
                   '\n'
                   'You can use a portable ram to break down doors. W'},
    {   'answer': 'twenty',
        'context': 'This wooden case can hold up to twenty crossbow bolts\n'
                   'Case, Map or Scroll.\n'
                   '\n'
                   'This cylindrical leather case can hold up to ten rolled-up '
                   'sheets of

In [14]:
prediction = pipe.run(
    query="What can I use Charisma for?",
    params={
        "Retriever": {"top_k": 10},
        "Reader": {"top_k": 5}
    }
)
from haystack.utils import print_answers

print_answers(
    prediction,
    details="minimum" ## Choose from `minimum`, `medium`, and `all`
)

Inferencing Samples:   0%|          | 0/1 [00:00<?, ? Batches/s]

'Query: What can I use Charisma for?'
'Answers:'
[   {   'answer': 'Confidence, eloquence, leadership',
        'context': 'a bonus to Wisdom\n'
                   'Ability Score SummaryCharisma\n'
                   '\n'
                   'Measures: Confidence, eloquence, leadership\n'
                   '\n'
                   'Important for: Bard, sorcerer, warlock\n'
                   '\n'
                   'Races that have '},
    {   'answer': 'Deception, Intimidation, Performance, Persuasion',
        'context': ', Insight, Medicine, Perception, Survival\n'
                   'Charisma\tDeception, Intimidation, Performance, '
                   'Persuasion\n'
                   '\n'
                   'Sometimes, the DM might ask for an ability check '},
    {   'answer': 'interact effectively with others',
        'context': 's.\n'
                   'Charisma PHB p178[–]\n'
                   '\n'
                   'Charisma measures your ability to interact effectively '
